In [1]:
import time
notebookstart= time.time()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch

C:\Users\_v010ch_\Anaconda3\envs\uly22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
from typing import List, Tuple, Optional

import pandas as pd
import numpy as np

from PIL import Image
from pillow_heif import register_heif_opener
register_heif_opener() # for using Image.open for .heic without changes

from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
from itertools import product
#import multiprocessing as mp
import torch.multiprocessing as mp

In [6]:
#import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
%matplotlib inline

In [8]:
from src.mkcarfeatures import features as ft

In [9]:
DIR_DATA = os.path.join(os.getcwd(), 'data')
DIR_SUBM = os.path.join(os.getcwd(), 'subm')
DIR_SUBM_TRAIN = os.path.join(os.getcwd(), 'subm', 'train')
DIR_DATA_TRAIN = os.path.join(DIR_DATA, 'train')
DIR_DATA_TEST  = os.path.join(DIR_DATA, 'test')

In [10]:
test_img_names  = set(os.listdir(DIR_DATA_TEST))
train_img_names = set(os.listdir(DIR_DATA_TRAIN))
len(test_img_names), len(train_img_names)

(521, 530)

In [11]:
train_labels_df = pd.read_csv(os.path.join(DIR_DATA, 'train.csv'), sep=';', index_col=None)

In [12]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')
model.classes = [0, 2]  # person and car
_ = model.cpu()

Using cache found in C:\Users\_v010ch_/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-3 Python-3.10.4 torch-1.12.0 CUDA:0 (GeForce GTX 1050 Ti, 4096MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


In [13]:
from functools import partial

In [42]:
%%time 
with mp.Pool(processes = (mp.cpu_count() - 2)) as pool:
    #results = pool.map(ft.create_feeatures_mp, product(list(train_img_names)[:5], [DIR_DATA_TRAIN], [model]))
    #results = pool.map(ft.create_feeatures_mp, product(list(train_img_names)[:5], [DIR_DATA_TRAIN]))
    results = pool.map(partial(ft.create_feeatures_mp, inp_dir = DIR_DATA_TRAIN, inp_model = model), train_img_names)

CPU times: total: 500 ms
Wall time: 7min 17s


In [41]:
train_df = pd.DataFrame(results, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class', 'h', 'w'])
train_df = pd.merge(train_labels_df, train_df, how='left')
train_df.shape

(530, 10)

In [43]:
train_df[train_df.image_name == 'img_1890.jpg']

,image_name,distance,x_min,y_min,x_max,y_max,conf,class,h,w
147,img_1890.jpg,1.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
%%time 
with mp.Pool(processes = (mp.cpu_count() - 2)) as pool:
    #results = pool.map(ft.create_feeatures_mp, product(list(train_img_names)[:5], [DIR_DATA_TRAIN], [model]))
    #results = pool.map(ft.create_feeatures_mp, product(list(train_img_names)[:5], [DIR_DATA_TRAIN]))
    results = pool.map(partial(ft.create_feeatures_mp, inp_dir = DIR_DATA_TEST, inp_model = model), test_img_names)

CPU times: total: 688 ms
Wall time: 7min 15s


In [46]:
test_df = pd.DataFrame(results, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class', 'h', 'w'])
test_df.shape

(521, 9)

In [55]:
#test_df[test_df.image_name == 'img_2571.jpg']
#test_df.head()

yolov5 не найдено машин:   
train: img_1890.jpg,    
test: img_1888.jpg, img_2674.heic, img_2571.jpg, img_1889.jpg(only person)   

In [56]:
sns.histplot(train_df, x='h')
plt.show()

C:\Users\_v010ch_\AppData\Local\Temp\ipykernel_5552\4050404586.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [57]:
sns.histplot(train_df, x='w')
plt.show()

C:\Users\_v010ch_\AppData\Local\Temp\ipykernel_5552\636209886.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [58]:
train_df['class'].value_counts()

2.0    528
0.0      1
Name: class, dtype: int64

In [59]:
test_df['class'].value_counts()

2.0    517
0.0      1
Name: class, dtype: int64

In [60]:
for el in ['x_min', 'y_min', 'x_max', 'y_max', 'h', 'w']:
    train_df[f'log_{el}'] = train_df[el].apply(lambda x: np.log(x))
    test_df[f'log_{el}'] = test_df[el].apply(lambda x: np.log(x))

In [61]:
train_df.head(20)

,image_name,distance,x_min,y_min,x_max,...,log_y_min,log_x_max,log_y_max,log_h,log_w
0,img_1596.jpg,4.88,1661.424072,1172.543701,2280.588135,...,7.066931,7.732189,7.448828,6.301344,6.428370
1,img_1600.jpg,1.54,1125.089722,1185.073486,3012.445312,...,7.077560,8.010507,7.907166,7.334073,7.542932
2,img_1601.jpg,3.68,2442.252441,1335.316040,3050.830566,...,7.196923,8.023169,7.488585,6.114136,6.411125
3,img_1603.jpg,2.22,2602.999512,1277.598389,3310.421875,...,7.152737,8.104831,7.522870,6.349610,6.561628
4,img_1605.jpg,3.73,2487.154053,1227.636475,3107.961426,...,7.112846,8.041722,7.428144,6.120398,6.431021
5,img_1606.jpg,4.52,3095.597412,1149.528809,4031.594238,...,7.047107,8.301917,7.653618,6.865610,6.841612
6,img_1607.jpg,5.40,2974.618896,1361.916992,4026.968750,...,7.216649,8.300769,7.757575,6.884802,6.958781
7,img_1608.jpg,6.21,2847.306641,1254.970581,4028.884766,...,7.134867,8.301245,7.716377,6.897546,7.074606
8,img_1612.jpg,7.12,2691.801270,1402.035400,4027.101074,...,7.245680,8.300802,7.719304,6.744478,7.196911
9,img_1613.jpg,5.26,2883.218018,1201.698486,4029.379639,...,7.091491,8.301368,7.743700,7.007866,7.044174


In [62]:
train_df.to_csv(os.path.join(DIR_DATA, 'train_upd.csv'), index = False)
test_df.to_csv(os.path.join(DIR_DATA, 'test_upd.csv'), index = False)